In [1]:
# Assign a friendly name to your network and snapshot
%run startup.py
load_questions()

# Initialize a network and a snapshot
bf_set_network("network-example-filters")

SNAPSHOT_NAME = "mycurrent"
SNAPSHOT_PATH = "networks/example-filters/mycurrent"
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf_upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf_upload_diagnostics)


'mycurrent'

In [2]:
load_questions()
#?bfq.nodeProperties

In [3]:
parse_status = bfq.fileParseStatus().answer().frame()

In [4]:
parse_status

,File_Name,Status,Nodes
0,configs/ASA_after_acl_summary.cfg,PARTIALLY_UNRECOGNIZED,['asa_after_acl_summary']
1,configs/ASA_before_acl_summary.cfg,PARTIALLY_UNRECOGNIZED,['asa_before_acl_summary']
2,configs/N9kv-1.cfg,PARTIALLY_UNRECOGNIZED,['n9kv-1']
3,configs/N9kv-2.cfg,PARTIALLY_UNRECOGNIZED,['n9kv-2']
4,configs/after_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['after_summary_asa']
5,configs/before_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['before_summary_asa']
6,configs/ciscoasa.cfg,PARTIALLY_UNRECOGNIZED,['ciscoasa']


In [5]:
question2 = bfq.referencedStructures(nodes='ciscoasa').answer().frame()
question2

,Structure_Type,Structure_Name,Context,Source_Lines
0,class-map,inspection_default,policy-map class,configs/ciscoasa.cfg:[1304]
1,interface,GigabitEthernet0/0,interface,configs/ciscoasa.cfg:[16]
2,interface,GigabitEthernet0/1,interface,configs/ciscoasa.cfg:[21]
3,interface,GigabitEthernet0/2,interface,configs/ciscoasa.cfg:[26]
4,interface,GigabitEthernet0/3,interface,configs/ciscoasa.cfg:[32]
5,interface,GigabitEthernet0/4,interface,configs/ciscoasa.cfg:[40]
6,interface,GigabitEthernet0/5,interface,configs/ciscoasa.cfg:[48]
7,interface,GigabitEthernet0/6,interface,configs/ciscoasa.cfg:[56]
8,interface,GigabitEthernet0/7,interface,configs/ciscoasa.cfg:[64]
9,interface,GigabitEthernet0/8,interface,configs/ciscoasa.cfg:[72]


In [26]:
question2 = bfq.referencedStructures(nodes='n9kv-1').answer().frame()
question2

TypeError: 'module' object is not callable

In [6]:
'''
出力結果の見方:
Line_Contentはどのアクセスリストがその通信を許可しているのか表示
'''

src = []
ip_flow = HeaderConstraints(srcIps='10.1.0.1',
                           dstIps='255.255.255.0')
answer = bfq.testFilters(headers=ip_flow,
                           nodes='before_summary_asa',
                           filters="SPLIT-ACL").answer() 
show(answer.frame())

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,before_summary_asa,SPLIT-ACL,Start Location: before_summary_asaSrc IP: 10.1.0.1Src Port: 49152Dst IP: 255.255.255.0Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


In [11]:
import json
import csv

src = []
ip_flow = HeaderConstraints(srcIps='192.168.1.1')
                           #dstIps='255.255.255.0')
answer = bfq.testFilters(headers=ip_flow,
                           nodes='after_summary_asa',
                           filters="SPLIT-ACL").answer() 
show(answer.frame())

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,after_summary_asa,SPLIT-ACL,Start Location: after_summary_asaSrc IP: 192.168.1.1Src Port: 49152Dst IP: 8.8.8.8Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


<h1>CSVからACLのテストをする</h1>

In [16]:
import json
import csv

src = []

with open('ASA_acl_test.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        src = row.get('src')
        dest = row.get('dest')
        port = row.get('port')
        node = row.get('node')
        filter_name = row.get('filter')
        
        ip_flow = HeaderConstraints(srcIps=src,
                                             dstIps=dest)
        answer = bfq.testFilters(headers=ip_flow,
                                   nodes=node).answer().frame()

        show(answer)
    f.close()

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,ciscoasa,dsi_split_tunnel,Start Location: ciscoasaSrc IP: 172.20.0.2Src Port: 49152Dst IP: 192.168.1.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'dsi_split_tunnel'"", lineDescription=None)])"
1,ciscoasa,outside_cryptomap_HPS-CHI,Start Location: ciscoasaSrc IP: 172.20.0.2Src Port: 49152Dst IP: 192.168.1.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'extended ipv4 access-list' named 'outside_cryptomap_HPS-CHI'"", lineDescription=None)])"
2,ciscoasa,~ZONE_OUTGOING_ACL~SECURITY_LEVEL_52~,Start Location: ciscoasaSrc IP: 172.20.0.2Src Port: 49152Dst IP: 192.168.1.1Dst Port: 80IP Protocol: TCP,PERMIT,Allow traffic originating from this device,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by filter named ~ZONE_OUTGOING_ACL~SECURITY_LEVEL_52~, index 0: Allow traffic originating from this device', lineDescription='Allow traffic originating from this device')])"
3,ciscoasa,~SERVICE_OBJECT~service-dns-dst~,Start Location: ciscoasaSrc IP: 172.20.0.2Src Port: 49152Dst IP: 192.168.1.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'object service' named 'service-dns-dst'"", lineDescription=None)])"
4,ciscoasa,~SERVICE_OBJECT~icmpecho1~,Start Location: ciscoasaSrc IP: 172.20.0.2Src Port: 49152Dst IP: 192.168.1.1Dst Port: 80IP Protocol: TCP,PERMIT,line:0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by object service named icmpecho1, index 0: IpAccessListLine{action=PERMIT, matchCondition=MatchHeaderSpace{headerSpace=HeaderSpace{}}}', lineDescription='IpAccessListLine{action=PERMIT, matchCondition=MatchHeaderSpace{headerSpace=HeaderSpace{}}}')])"
5,ciscoasa,~SERVICE_OBJECT~object-srv2~,Start Location: ciscoasaSrc IP: 172.20.0.2Src Port: 49152Dst IP: 192.168.1.1Dst Port: 80IP Protocol: TCP,PERMIT,line:0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by object service named object-srv2, index 0: IpAccessListLine{action=PERMIT, matchCondition=MatchHeaderSpace{headerSpace=HeaderSpace{}}}', lineDescription='IpAccessListLine{action=PERMIT, matchCondition=MatchHeaderSpace{headerSpace=HeaderSpace{}}}')])"
6,ciscoasa,~SERVICE_OBJECT_GROUP~Covance_Allow_GRP~,Start Location: ciscoasaSrc IP: 172.20.0.2Src Port: 49152Dst IP: 192.168.1.1Dst Port: 80IP Protocol: TCP,PERMIT,line:0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by object-group service named Covance_Allow_GRP, index 0: IpAccessListLine{action=PERMIT, matchCondition=OrMatchExpr{disjuncts=[MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[12345,12345]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[4444,4444]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[1234,1234]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[443,443]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[80,80]], ipProtocols=[TCP]}}]}}', lineDescription='IpAccessListLine{action=PERMIT, matchCondition=OrMatchExpr{disjuncts=[MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[12345,12345]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[4444,4444]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[1234,1234]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[443,44

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,ciscoasa,inside_access_in,Start Location: ciscoasaSrc IP: 172.20.0.3Src Port: 49152Dst IP: 10.0.0.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'extended ipv4 access-list' named 'inside_access_in'"", lineDescription=None)])"
1,ciscoasa,~SERVICE_OBJECT_GROUP~Object-GRP-6~,Start Location: ciscoasaSrc IP: 172.20.0.3Src Port: 49152Dst IP: 10.0.0.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'object-group service' named 'Object-GRP-6'"", lineDescription=None)])"
2,ciscoasa,~SERVICE_OBJECT_GROUP~network_monitoring~,Start Location: ciscoasaSrc IP: 172.20.0.3Src Port: 49152Dst IP: 10.0.0.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'object-group service' named 'network_monitoring'"", lineDescription=None)])"
3,ciscoasa,~PROTOCOL_OBJECT_GROUP~Object-GRP-7~,Start Location: ciscoasaSrc IP: 172.20.0.3Src Port: 49152Dst IP: 10.0.0.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'object-group protocol' named 'Object-GRP-7'"", lineDescription=None)])"
4,ciscoasa,dsi_split_tunnel_1,Start Location: ciscoasaSrc IP: 172.20.0.3Src Port: 49152Dst IP: 10.0.0.1Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'dsi_split_tunnel_1'"", lineDescription=None)])"
5,ciscoasa,~SERVICE_OBJECT_GROUP~AIGHCRS01_SRV~,Start Location: ciscoasaSrc IP: 172.20.0.3Src Port: 49152Dst IP: 10.0.0.1Dst Port: 80IP Protocol: TCP,PERMIT,line:0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by object-group service named AIGHCRS01_SRV, index 0: IpAccessListLine{action=PERMIT, matchCondition=OrMatchExpr{disjuncts=[MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[43856,43856]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[8088,8088]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[8080,8080]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[4033,4033]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[3389,3389]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[2383,2383]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[2382,2382]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[1434,1434]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[1433,1433]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[443,443]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[135,135]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[80,80]], ipProtocols=[TCP]}}]}}', lineDescription='IpAccessListLine{action=PERMIT, matchCondition=OrMatchExpr{disjuncts=[MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[43856,43856]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[8088,8088]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[8080,8080]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[4033,4033]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[3389,3389]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[2383,2383]], ipProtocols=[TCP]}}, MatchHeaderSpace{headerSpace=HeaderSpace{dstPorts=[[2382,2382]

In [17]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import os
import subprocess
from pybatfish.client.commands import *
from pybatfish.question.question import load_questions, list_questions
from pybatfish.question import bfq

'''
出力結果の見方:
Line_Contentはどのアクセスリストがその通信を許可しているのか表示
'''

src = []
ip_flow = HeaderConstraints(srcIps='10.1.0.1',
                           dstIps='255.255.255.0')
answer = bfq.testFilters(headers=ip_flow,
                           nodes='before_summary_asa',
                           filters="SPLIT-ACL").answer() 
show(answer.frame())



def main():
  os.system('python setup.py')
  load_questions()
  
  # Initialize a network and a snapshot
  bf_set_network("network-example-filters")
  
  SNAPSHOT_NAME = "mycurrent"
  SNAPSHOT_PATH = "networks/example-filters/mycurrent"
  bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)
  
  parse_warning = bfq.parseWarning().answer().frame()
  parse_warning
  
  parse_status = bfq.fileParseStatus().answer().frame()
  parse_status

if __name__ == '__main__':
    main()

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,before_summary_asa,SPLIT-ACL,Start Location: before_summary_asaSrc IP: 10.1.0.1Src Port: 49152Dst IP: 255.255.255.0Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf_upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf_upload_diagnostics)


In [30]:
ip_owners_ans = bfq.ipOwners().answer(

TypeError: 'module' object is not callable